This notebook is for Q learning. The multi-timestep should be taken better care of under this case. Train on batch multiple TD may not be a good idea in this case.

In [1]:
import os
# disable tensorflow logging
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import statistics
import numpy as np
import tensorflow as tf
import gym
import tqdm
import collections


2023-03-23 16:03:04.319007: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-23 16:03:04.400651: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-23 16:03:04.400666: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-23 16:03:04.781331: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [2]:
class Env():
    def __init__(self, max_step=1000):
        self.env = gym.make("CartPole-v1")
        self.max_step = max_step
        self.cur_step = 1
        self.action_space = self.env.action_space
        
    def reset(self):
        self.cur_step = 1
        initial_state, info = self.env.reset()
        initial_state = self.add_step_into_state(initial_state)
        return initial_state, info
    
    def add_step_into_state(self, state):
#         state = np.concatenate([state, np.array([self.cur_step/self.max_step])])
        
        return state
    
    def step(self, action):
        self.cur_step += 1
        state, reward, done, _, _ = self.env.step(action)
        if self.cur_step > self.max_step:
            reward = 1.0
            done = True
            self.cur_step = self.max_step + 1
        else:
            if done:
                reward = 1.0
        state = self.add_step_into_state(state)
        return state, reward, done, _, _

In [3]:
env = gym.make("CartPole-v1")
# env = Env()
initial_state, _ = env.reset()
initial_state_shape = initial_state.shape
action_space = env.action_space.n
eps = np.finfo(np.float32).eps.item()
gamma = 0.99
lr = 0.0003
step_length = 50
use_dueling = True 
"""Q learning training is much harder than policy gradient"""

if use_dueling:
    lr = 0.00001
    gamma = 0.99

# define model

In [4]:
"""
The model will use basic Q-learning
"""

def get_model():
    inputs = tf.keras.layers.Input(shape=initial_state_shape)
    hidden = tf.keras.layers.Dense(64, activation="relu")(inputs)
    hidden = tf.keras.layers.Dense(128, activation="relu")(hidden)
    outs = tf.keras.layers.Dense(action_space, activation=None)(hidden)
    return tf.keras.Model(inputs, outs)

def get_dueling_model():
    """
    A = Q - S
    Q = A + S - mean(A)
    """
    inputs = tf.keras.layers.Input(shape=initial_state_shape)
    hidden = tf.keras.layers.Dense(64, activation="relu")(inputs)
    hidden = tf.keras.layers.Dense(128, activation="relu")(hidden)
    A = tf.keras.layers.Dense(action_space, activation=None)(hidden)
    S = tf.keras.layers.Dense(1, activation=None)(hidden)
    A_mean = tf.math.reduce_mean(A, axis=1, name="mean")
    outs = tf.keras.layers.Add(name="outs")([A, S, -A_mean])
    
    return tf.keras.Model(inputs, outs)
    
if use_dueling:
    model = get_dueling_model()
else:
    model = get_model()
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 4)]          0           []                               
                                                                                                  
 dense (Dense)                  (None, 64)           320         ['input_1[0][0]']                
                                                                                                  
 dense_1 (Dense)                (None, 128)          8320        ['dense[0][0]']                  
                                                                                                  
 dense_2 (Dense)                (None, 2)            258         ['dense_1[0][0]']                
                                                                                              

2023-03-23 16:03:05.337520: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-23 16:03:05.337715: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-23 16:03:05.337752: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2023-03-23 16:03:05.337779: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2023-03-23 16:03:05.337805: W tensorflow/c

# define data collection

In [5]:
def _run_step_numpy(action):
    state, reward, done, _, _ = env.step(action)
    return (state.astype(np.float32), np.array(reward, dtype=np.float32), np.array(done, dtype=np.int32))

def run_step_tf(action):
    return tf.numpy_function(_run_step_numpy, [action], (tf.float32, tf.float32, tf.int32))

def run_step(start_state, model, step_length):

    values = tf.TensorArray(tf.float32, size=0, dynamic_size=True)
    actions = tf.TensorArray(tf.int32, size=0, dynamic_size=True)
    rewards = tf.TensorArray(tf.float32, size=0, dynamic_size=True)
    done = tf.constant(True, dtype=tf.bool)
    done_shape = done.shape
    
    state = start_state
    for t in tf.range(step_length):
        q_output = model(tf.expand_dims(state, 0))
        q_output = tf.squeeze(q_output)
        action = tf.math.argmax(q_output, output_type=tf.int32)
        
        value = q_output[action]
        state, reward, done = run_step_tf(action)
        state.set_shape(initial_state_shape)
        
        actions = actions.write(t, action)
        values = values.write(t, tf.squeeze(value))
        rewards = rewards.write(t, reward)
        done = tf.cast(done, dtype=tf.bool)
        done.set_shape(done_shape)
        if done:
            break
    next_value = model(tf.expand_dims(state, 0))
    next_value = tf.squeeze(next_value)
    next_value = tf.math.reduce_max(next_value)
    
     
    actions = actions.stack()
    values = values.stack()
    rewards = rewards.stack()
    return values, actions, rewards, next_value, state, done

In [6]:
state_, _ = env.reset()
result = run_step(state_, model, 100)
values, actions, rewards, next_value, state, done = result

/home/wei/anaconda3/envs/tf_211/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


# define returns

In [7]:
def get_returns(rewards_array, gamma=0.99):
    """
    Input: total_rewards is a value
    Output: 
        discount_array: array of discount rate.
            index i to the (end timestamp + 1) discount rate
        returns: array of discounted returns
            index i means the returns between index i to the index(end timestamp + 1)
    """
    rewards_array = tf.squeeze(rewards_array)
    gamma = tf.constant(gamma, tf.float32)
    discounted_return = tf.constant(0.0, tf.float32)
    dshape = discounted_return.shape
    returns = tf.TensorArray(tf.float32, size=0, dynamic_size=True)
    for idx in tf.range(tf.shape(rewards_array)[0] -1, -1, -1):
        discounted_return = gamma * discounted_return + rewards_array[idx]
        discounted_return.set_shape(dshape)
        returns = returns.write(idx, discounted_return)
    discount_array = tf.ones_like(rewards_array, dtype=tf.float32) * gamma
    discount_array = tf.math.cumprod(discount_array, reverse=True)
    returns = returns.stack()
    
    returns = (returns - tf.math.reduce_mean(returns)) / (eps + tf.math.reduce_std(returns))
    
    return discount_array, returns
 

In [8]:
rewards = result[2]
get_returns(rewards)

(<tf.Tensor: shape=(48,), dtype=float32, numpy=
 array([0.61729044, 0.6235257 , 0.6298239 , 0.63618577, 0.64261186,
        0.64910287, 0.65565944, 0.6622822 , 0.66897196, 0.6757292 ,
        0.6825548 , 0.68944925, 0.6964134 , 0.7034479 , 0.7105534 ,
        0.7177307 , 0.7249805 , 0.7323035 , 0.7397005 , 0.74717224,
        0.75471944, 0.7623429 , 0.7700433 , 0.77782154, 0.7856783 ,
        0.79361445, 0.80163074, 0.809728  , 0.8179071 , 0.8261688 ,
        0.8345139 , 0.8429433 , 0.8514579 , 0.8600585 , 0.8687459 ,
        0.8775211 , 0.88638496, 0.89533836, 0.90438217, 0.91351736,
        0.9227448 , 0.9320654 , 0.9414802 , 0.95099014, 0.9605961 ,
        0.97029907, 0.98010004, 0.99      ], dtype=float32)>,
 <tf.Tensor: shape=(48,), dtype=float32, numpy=
 array([ 1.5685304 ,  1.5116849 ,  1.4542651 ,  1.3962656 ,  1.3376802 ,
         1.2785032 ,  1.2187283 ,  1.1583496 ,  1.0973611 ,  1.0357565 ,
         0.97352946,  0.910674  ,  0.84718364,  0.78305197,  0.71827245,
         0.

# Loss

In [9]:
loss_func = tf.keras.losses.MeanSquaredError(reduction=tf.keras.losses.Reduction.SUM_OVER_BATCH_SIZE)
def calculate_loss(returns, values, value_next, discount_array):
    """
    Policy part ---------
    V(St) = E(Q) = Pi(St, At1; theta) * Q(St, At1) + ...
    G denote gradient W.R.T theta
    
    
    G(V(St)) = G[Pi(St, At1; theta) * Q(St, At1) + ...]
                approximate= Pi(St, A1)* G(logpi(St, At1; theta) * Q) + ...  # chain rule G(logpi) = 1/pi * G(pi)
                = E[ G(logpi * Q) ] # Pi(St, A) is the PDF, so this is the expectation
              [1]  approximate= G(logpi * Q)  # monte carlo approximation
              [2]  = G(logpi * (Q - baseline))  where baseline can be V. This is the A2C
                  Qt can be approximate by Yt
              Yt = gamma^T * Q(T) + r + gamma*r + gamma^2*r + ...
              
    Critic Part TD learning -----------
    Qt = discounted_ovserved + QT
            
    """
    values = tf.squeeze(values)
    Yt = tf.squeeze(returns) + discount_array * value_next
    
    loss = loss_func(tf.expand_dims(values, 1), tf.expand_dims(Yt,1))
    
    return loss#, abs(Yt - values)

In [10]:
values, actions, rewards, next_value, state, done = result
discount_array, returns_array = get_returns(rewards, gamma)
calculate_loss(returns_array, values, next_value, discount_array)

<tf.Tensor: shape=(), dtype=float32, numpy=0.99447125>

# train step

In [11]:
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)

@tf.function
def run_train_step(model, optimizer, start_state, step_length):
    with tf.GradientTape() as tape:
        STEP_RES = run_step(start_state, model, step_length)
        values, actions, rewards, next_value, state, done = STEP_RES
        discount_array, returns_array = get_returns(rewards, gamma)
        loss = calculate_loss(returns_array, values, next_value, discount_array)
    gradient = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradient, model.trainable_variables))
    return STEP_RES, loss

In [12]:
_state, _ = env.reset()
run_train_step(model, optimizer, _state, 50)

((<tf.Tensor: shape=(28,), dtype=float32, numpy=
  array([0.00719846, 0.00546249, 0.00754609, 0.00549177, 0.00784724,
         0.00557674, 0.0085881 , 0.00587735, 0.00985033, 0.00628006,
         0.01065368, 0.00670577, 0.01065022, 0.00744282, 0.0106605 ,
         0.00854854, 0.01098789, 0.00979421, 0.01157524, 0.01098666,
         0.01211687, 0.01234939, 0.01250588, 0.01354526, 0.01299097,
         0.01507375, 0.01358637, 0.01682195], dtype=float32)>,
  <tf.Tensor: shape=(28,), dtype=int32, numpy=
  array([0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1,
         0, 1, 0, 1, 0, 1], dtype=int32)>,
  <tf.Tensor: shape=(28,), dtype=float32, numpy=
  array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], dtype=float32)>,
  <tf.Tensor: shape=(), dtype=float32, numpy=0.014323888>,
  <tf.Tensor: shape=(4,), dtype=float32, numpy=array([-0.02062696,  0.01342651,  0.21177892,  0.70574605], dtype=float32)>

# RUN

In [13]:
min_epoch = 100
max_epoch = 10000
step_length = 50
thred = 475
running_rewards = collections.deque(maxlen=100)
max_steps_per_epoch = 500
all_rewards = []
all_running_rewards = []
t = tqdm.trange(max_epoch)
for i in t:
    restart = False
    start_state, _ = env.reset()
    cur_step = 0
    epoch_reward = 0
    while cur_step < max_steps_per_epoch:
        try:
            STEP_RES, loss= run_train_step(model, optimizer, start_state, step_length)
        except Exception:
            restart = True
            break
        cur_step += step_length
        values, actions, rewards, next_value, state, done = STEP_RES
        epoch_reward += int(sum(rewards))
        if done:
            break
    if restart:
        continue
    running_rewards.append(epoch_reward)
    avg_reward = statistics.mean(running_rewards)
    all_rewards.append(epoch_reward)
    all_running_rewards.append(avg_reward)
    t.set_postfix(running_rewards=avg_reward, current_reward=epoch_reward, loss=float(loss))
    if avg_reward > thred and i > min_epoch:
        break
    
    


 19%|█▊        | 1853/10000 [01:12<05:20, 25.41it/s, current_reward=69, loss=0.891, running_rewards=38.8]

KeyboardInterrupt



In [ ]:
from matplotlib import pyplot as plt

In [ ]:
_state, _ = env.reset()
result = run_step(_state, model, 1000)
values, actions, rewards, next_value, state, done = result

In [ ]:
plt.plot(range(len(values)), values)

In [ ]:
# Render an episode and save as a GIF file

from IPython import display as ipythondisplay
from PIL import Image

render_env = gym.make("CartPole-v1", render_mode='rgb_array')

def render_episode(env: gym.Env, model: tf.keras.Model, max_steps: int): 
  state, info = env.reset()
  state = tf.constant(state, dtype=tf.float32)
  screen = env.render()
  images = [Image.fromarray(screen)]
  values = []

  for i in range(1, max_steps + 1):
    state = tf.expand_dims(state, 0)
    value_output = model(state)
    value_output = np.squeeze(value_output)
    action = np.argmax(np.squeeze(value_output))
    values.append(value_output[action])

    state, reward, done, truncated, info = env.step(action)
    state = tf.constant(state, dtype=tf.float32)

    # Render screen every 10 steps
    if i % 10 == 0:
      screen = env.render()
      images.append(Image.fromarray(screen))

    if done:
      break

  return images, values


# Save GIF image
images,values = render_episode(render_env, model, max_steps_per_epoch)
image_file = 'cartpole-v1.gif'
# loop=0: loop forever, duration=1: play each frame for 1ms
images[0].save(
    image_file, save_all=True, append_images=images[1:], loop=0, duration=1)


In [ ]:
import tensorflow_docs.vis.embed as embed
embed.embed_file(image_file)

In [ ]:
values = tf.stack(values)
values = values.numpy()
plt.plot(range(len(values)), values)